# Machine Learning para imóveis
Vamos fazer um programa para auxiliar novos inscritos a definirem valor de seus imóveis antes de anunciarem. Bastará informar as características do imóvel que pretende ser vendido que o programa recomendará o valor.

### Importar bibliotecas necessárias

In [ ]:
import os
import pandas as pd
import numpy as np
import collections as clt

### Definindo caminhos e arquivos

In [ ]:
git_path = "/".join( os.getcwd().split("/")[:-1] )
data_path = git_path + "/data/"

### Importação da base de dados

In [ ]:
imovel_file = "imovel_csv.csv"
imovel_df = pd.read_csv(data_path+imovel_file, sep=";")
imovel_df.head(3)

### Verificando dados faltantes

In [ ]:
imovel_df.count()

### Removendo linhas que não tem CEP

In [ ]:
imovel_df = imovel_df[np.isfinite( imovel_df["cep"]) ]
imovel_df["cep"] = imovel_df["cep"].astype(int)
imovel_df.head(3)

### Importação da base de ceps

In [ ]:
cep_file = "cep_sp.csv"
cep_df = pd.read_csv(data_path+cep_file, sep=",")
cep_df["cep"] = cep_df["cep"].astype(int)
cep_df.head(3)

### Join entre as bases

In [ ]:
imovel_df = pd.merge(left=imovel_df, right=cep_df[["cep", "bairro"]], how="left", on="cep", copy=False)

### Tratamento das características

In [ ]:
carac_list = []
for i in imovel_df["caracteristicas"]:
    if type(i) == str:
        carac_list += i.lower().replace(" ", "").split(",")

In [ ]:
carac_serie = pd.Series( clt.Counter( carac_list ) ).sort_values()
carac_serie / imovel_df.shape[0]
carac_serie = carac_serie[carac_serie>100  ]

In [ ]:
def carac_column( x, carac_str ):
    try:
        if carac_str in x:
            return 1
        else:
            return 0
    except TypeError as err:
        return 0

    
imovel_df["new_caracteristicas"] = imovel_df["caracteristicas"].apply(lambda x: x.lower().replace(" ", "") if type(x)==str else np.nan)

for i in carac_serie.index:
    imovel_df[i] = imovel_df["new_caracteristicas"].apply( func=carac_column, args=[i] )    
    

In [ ]:
imovel_df.head(3)

### Definição da tabela de modelagem

In [ ]:
colunas_tt = list( imovel_df.columns )

In [ ]:
colunas_modelo = ['cep','latitude', 'longitude',
                  'qtde_banheiros', 'qtde_quartos', 'qtde_vagas']

colunas_modelo += colunas_tt[ colunas_tt.index("new_caracteristicas")+1:]
colunas_modelo += ["valor_venda"]

In [ ]:
tb_model = imovel_df[colunas_modelo].copy()
tb_model.head()